<a href="https://colab.research.google.com/github/dendej/AI-for-Trading/blob/master/EA31337_Interactive_Backtester_(v3)_%5Bv20240422%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EA31337 Interactive Backtester

This [Colab](https://colab.research.google.com/notebooks/intro.ipynb) aims at backtesting [EA31337 trading robot](https://github.com/EA31337/EA31337).

- To use this notebook interactively, log in using your Google account, then follow steps found below.

- The test is split into smaller sections, for each you have to hit the play button (►️) to execute it.

- By default the test is executed on a hosted runtime, however it is possible to connect to your local runtime as well.

- If you've any issues or suggestions, feel free to leave the comments.

In [ ]:
# @title 1. Install EA tester. { display-mode: "form" }
# @markdown ⬅ Hit the play button on the left to install EA tester and dependencies.
# @markdown Once completed, please move to the next section.
!pip install ansible
!ansible-galaxy install --force-with-deps git+https://github.com/EA31337/ansible-role-mt-runner.git,dev
!ansible-galaxy role list
!echo EA tester has been installed!
!echo You can now run the next section.

In [ ]:
%%shell
# @title 2. Run backtest. { display-mode: "form" }
# @markdown ⬅ Choose configuration below and hit the play button on the left to run the backtest.
EA31337_MODE="Advanced" # @param ["Lite", "Advanced", "Rider", "Elite"]
EA31337_VERSION=v3.000-rc1 # @param ["v3.000-rc1"]
TEST_DATE_FROM="2022-01-01" # @param {type:"date"}
TEST_DATE_TO="2024-01-31" # @param {type:"date"}
TEST_DEPOSIT=1000 # @param {type:"slider", min:100, max:100000, step:100}
TEST_LEVERAGE="1:500" # @param ["1:100", "1:200", "1:500"]
TEST_SYMBOL="EURUSD" # @param ["EURUSD", "EURGBP"]
export EA31337_MODE EA31337_VERSION TEST_DATE_FROM TEST_DATE_TO TEST_DEPOSIT TEST_LEVERAGE TEST_SYMBOL
# @markdown Notes:
# @markdown - The backtest run can take a while, depending on the selected period.
# @markdown - Consequent runs should be much quicker.
# @markdown - Once execution is successful, please execute the next section to display the results.
# Run test.
cat <<EOF | ANSIBLE_STDOUT_CALLBACK=yaml ansible-playbook -i "$(hostname)," -c local /dev/stdin && echo The backtest has been completed!
---
- hosts: localhost
  tasks:
    - name: Run backtest
      ansible.builtin.import_role:
        name: ansible-role-mt-runner
      vars:
        mt_runner_bt_mt5_config:
          Common:
            Login: '{{ "MTAwMDI0NjY2Mjk=" | b64decode }}'
            Password: '{{ "VXMhdEowR2w=" | b64decode }}'
            Server: MetaQuotes-Demo
          Tester:
            Deposit: "{{ ansible_facts.env.TEST_DEPOSIT }}"
            ExecutionMode: 0
            Expert: EA31337-{{ ansible_facts.env.EA31337_MODE }}-{{ ansible_facts.env.EA31337_VERSION }}.ex5
            FromDate: "{{ ansible_facts.env.TEST_DATE_FROM }}"
            Leverage: "{{ ansible_facts.env.TEST_LEVERAGE }}"
            Login: 12345
            Model: 1
            Period: M1
            Report: report.html
            ReplaceReport: 1
            ShutdownTerminal: 1
            Symbol: "{{ ansible_facts.env.TEST_SYMBOL }}"
            ToDate: "{{ ansible_facts.env.TEST_DATE_TO }}"
        mt_runner_get_url_list:
        - dest: "{{ mt_runner_mt_path }}/MQL5/Experts"
          url: https://github.com/EA31337/EA31337/releases/download/{{ ansible_facts.env.EA31337_VERSION }}/EA31337-{{ ansible_facts.env.EA31337_MODE }}-{{ ansible_facts.env.EA31337_VERSION }}.ex5
EOF

In [ ]:
# @title 3. Display backtest results
# @markdown ⬅ Hit the play button to display or refresh the backtest results. { display-mode: "form" }
import glob, os
from IPython.display import HTML, Image, display
os.chdir(glob.glob('/root/.wine/drive_c/Program Files/MetaTrader*')[0])
for file in os.listdir('.'):
    if file.endswith('.png'):
      display(Image(filename=file))
HTML(data=open("report.html", mode='r', encoding='utf-16').read())